In [ ]:
!pip install langchain langchain-core langchain-community
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
 

In [ ]:
import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-4o")

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size = 500,
                    chunk_overlap = 50,
                    length_function = tiktoken_len
)

docs = text_splitter.split_documents(pages)

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

<ipython-input-5-074fe8ca1bc4>:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, hf, persist_directory="./chroma_db")

In [ ]:
query = "장기저탄소발전전략"
results = db.similarity_search(query)

print(results[0].page_content)
print("-" * 100)
print(results[1].page_content)
print("-" * 100)
print(results[2].page_content)
print("-" * 100)
print(results[3].page_content)

02
※ ‘2050 탄소중립 추진전략’을 중심으로 작성된 이노비즈 정책브리프 2021-04호 이후 탄소중립 대응에 필요한 정책 기조와 글로벌 동향, 조사연구 결과 등을 꾸준히 소개할 예정입니다.
목 차탄소중립과 대응 필요성1탄소중립 대응을 위한 정부 정책3해외 주요국 동향6국내 대기업 동향7중소기업의 탄소중립 대응8부록: 주요 정부 정책과제 추진 일정10부록: 탄소중립 관련 정보 제공 누리집12
----------------------------------------------------------------------------------------------------
대비 2030년 46%, 2050년 80% 감축을 위한 목표 수립- 2020년 12월 2050년 온실가스 배출량 제로를 위해 해상풍력, 수소, 자동차 등 14개의 주요 분야 목표와 실행 계획이 포괄된 ‘그린성장전략’ 발표2)본 장은 한국무역협회에서 발행한 ‘주요국 탄소중립 정책과 시사점’(2021.4.12.)과 KOTRA 해외시장뉴스 ‘기후 정상회의 주요 내용 및 현지 반응’(2021.4.30.)의 주요 내용을 요약·재구성하여 작성함
----------------------------------------------------------------------------------------------------
억제하기 위해 노력하겠다는 것임- 모든 당사국은 2020년까지 장기저탄소발전전략(LEDS*)과 국가온실가스감축목표(NDC**)를 제출하기로 합의함 *LEDS : Long-term low greenhouse gas Emission Development Strategies (장기저탄소발전전략)**NDC : Nationally Determined Contribution (국가 온실가스 감축 목표, 5년 주기 제출)∙IPCC*는 2100년까지 지구 평균온도 상승폭을 1.5℃ 이내로 제한할 필요가 있다는 과학적 근거를 마련함*IPCC : Intergovernmental Panel on Clim

In [ ]:
results = db.similarity_search_with_relevance_scores(query, k=3)

<ipython-input-20-de85087c8bda>:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'page': 1, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='02\n※ ‘2050 탄소중립 추진전략’을 중심으로 작성된 이노비즈 정책브리프 2021-04호 이후 탄소중립 대응에 필요한 정책 기조와 글로벌 동향, 조사연구 결과 등을 꾸준히 소개할 예정입니다.\n목 차탄소중립과 대응 필요성1탄소중립 대응을 위한 정부 정책3해외 주요국 동향6국내 대기업 동향7중소기업의 탄소중립 대응8부록: 주요 정부 정책과제 추진 일정10부록: 탄소중립 관련 정보 제공 누리집12'), -216.02375539193943), (Document(metadata={'page': 8, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='대비 2030년 46%, 2050년 80% 감축을 위한 목표 수립- 2020년 12월 2050년 온실가스 배출량 제로를 위해 해상풍력, 수소, 자동차 등 14개의 주요 분야 목표와 실행 계획이 포괄된 ‘그린성장전략’ 발표2)본 장은 한국무역협회에서 발행한 ‘주요국 탄소중립 정책과 시사점’(2021.4.12.)과 KOTRA 해외시장뉴스 ‘기후 정상회의 주요 내용 및 현지 반응’(2021.4.30.)의 주요 내용을 요약·재구성하여 작성함'), -217.64892708092628), (Document(metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='억제하기 위해 노력하겠다는 것임- 모든 당사국은 2020년까지 장기저탄소발전전략(LEDS*)과 국가

In [ ]:
print(results[0][1], results[0][0].page_content)
print("-" * 100)
print(results[1][1], results[1][0].page_content)
print("-" * 100)
print(results[2][1], results[2][0].page_content)



-216.02375539193943 02
※ ‘2050 탄소중립 추진전략’을 중심으로 작성된 이노비즈 정책브리프 2021-04호 이후 탄소중립 대응에 필요한 정책 기조와 글로벌 동향, 조사연구 결과 등을 꾸준히 소개할 예정입니다.
목 차탄소중립과 대응 필요성1탄소중립 대응을 위한 정부 정책3해외 주요국 동향6국내 대기업 동향7중소기업의 탄소중립 대응8부록: 주요 정부 정책과제 추진 일정10부록: 탄소중립 관련 정보 제공 누리집12
----------------------------------------------------------------------------------------------------
-217.64892708092628 대비 2030년 46%, 2050년 80% 감축을 위한 목표 수립- 2020년 12월 2050년 온실가스 배출량 제로를 위해 해상풍력, 수소, 자동차 등 14개의 주요 분야 목표와 실행 계획이 포괄된 ‘그린성장전략’ 발표2)본 장은 한국무역협회에서 발행한 ‘주요국 탄소중립 정책과 시사점’(2021.4.12.)과 KOTRA 해외시장뉴스 ‘기후 정상회의 주요 내용 및 현지 반응’(2021.4.30.)의 주요 내용을 요약·재구성하여 작성함
----------------------------------------------------------------------------------------------------
-222.91854282979634 억제하기 위해 노력하겠다는 것임- 모든 당사국은 2020년까지 장기저탄소발전전략(LEDS*)과 국가온실가스감축목표(NDC**)를 제출하기로 합의함 *LEDS : Long-term low greenhouse gas Emission Development Strategies (장기저탄소발전전략)**NDC : Nationally Determined Contribution (국가 온실가스 감축 목표, 5년 주기 제출)∙IPCC*는 2100년까지 지구 평균온도 상승폭을 1.5℃ 이내로 제한

In [ ]:
db2 = Chroma(persist_directory="./chroma_db", embedding_function=hf)

<ipython-input-26-01cc4fa1af87>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db2 = Chroma(persist_directory="./chroma_db", embedding_function=hf)


In [ ]:
results = db2.similarity_search(query)
print(results[0].page_content)

02
※ ‘2050 탄소중립 추진전략’을 중심으로 작성된 이노비즈 정책브리프 2021-04호 이후 탄소중립 대응에 필요한 정책 기조와 글로벌 동향, 조사연구 결과 등을 꾸준히 소개할 예정입니다.
목 차탄소중립과 대응 필요성1탄소중립 대응을 위한 정부 정책3해외 주요국 동향6국내 대기업 동향7중소기업의 탄소중립 대응8부록: 주요 정부 정책과제 추진 일정10부록: 탄소중립 관련 정보 제공 누리집12


In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

import os

os.environ["OPENAI_API_KEY"] = "2sg8kxsseRytW3HOGXaGe1ESnMlAz9qGW1vpZ6EpkmQbCP2FfHdJJQQJ99BAACfhMk5XJ3w3AAAAACOGmcz2"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://youngwook-ai.openai.azure.com"

openai = AzureChatOpenAI(
              deployment_name="dev-gpt-4o-mini",
              temperature=0.3
)

In [ ]:
qa = RetrievalQA.from_chain_type(
        llm=openai,
        chain_type="stuff",
        retriever=db2.as_retriever(kwargs=),
        return_source_documents=True
)

In [ ]:
query = "탄소중립 사회란?"

In [ ]:
results = qa(query)

In [ ]:
print(results)

{'query': '탄소중립 사회란?', 'result': '탄소중립 사회란, 온실가스 배출량과 흡수량이 균형을 이루어 순 배출량이 0이 되는 사회를 의미합니다. 즉, 인간 활동으로 인해 발생하는 탄소 배출량을 줄이고, 이를 상쇄하기 위해 나무를 심거나 탄소를 흡수하는 기술을 활용하여 전체적인 탄소 배출을 줄이는 목표를 가지고 있습니다. 탄소중립 사회는 기후 변화에 대응하고 지속 가능한 발전을 이루기 위한 중요한 개념으로, 다양한 정책과 기술적 접근이 필요합니다.'}
